# 10. 트랜스포머로 만드는 대화형 챗봇 [프로젝트]

## 10-1. 프로젝트: 한국어 데이터로 챗봇 만들기

영어로 만들었던 챗봇을 한국어 데이터로 바꿔서 훈련시켜봅시다.

시작하기 전에 우선 주요 라이브러리 버전을 확인해 봅시다.

In [6]:
import tensorflow as tf
import tensorflow_datasets as tfds
import os
import re
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

print(tf.__version__)

2.13.0


### Step 1. 데이터 수집하기

한국어 챗봇 데이터는 송영숙님이 공개한 챗봇 데이터를 사용합니다.

이 데이터는 아래의 링크에서 다운로드할 수 있습니다.

songys/Chatbot_data

https://github.com/songys/Chatbot_data/blob/master/ChatbotData.csv

Cloud shell에서 아래 명령어를 입력해 주세요.

```bash
!mkdir -p aiffel/transformer_chatbot/data/
!ln -s ~/data/* ~/aiffel/transformer_chatbot/data/
```

## Step 2. 데이터 전처리하기

영어 데이터와는 전혀 다른 데이터인 만큼 영어 데이터에 사용했던 전처리와 일부 동일한 전처리도 필요하겠지만 전체적으로는 다른 전처리를 수행해야 할 수도 있습니다.

In [9]:
def preprocess_text(text):
    # Add space before punctuation
    text = re.sub(r"([?.!,])", r" \1 ", text)
    # Replace all characters except ".,!?" with space
    text = re.sub(r' +', " ", text)
    return text

def preprocess_csv_file(input_path, output_path):
    # Load the CSV file into a DataFrame
    data = pd.read_csv(input_path)
    
    # Apply preprocessing on 'Q' and 'A' columns
    data['Q'] = data['Q'].apply(preprocess_text)
    data['A'] = data['A'].apply(preprocess_text)

    # Drop the 'label' column
    data = data.drop(columns=['label'])
    
    # Save the preprocessed data to output file
    data.to_csv(output_path, index=False)

# Usage example:
input_filepath = 'aiffel/transformer_chatbot/data/ChatbotData.csv'
output_filepath = 'aiffel/transformer_chatbot/preprocessed/ChatbotData.csv'
preprocess_csv_file(input_filepath, output_filepath)


## Step 3. SubwordTextEncoder 사용하기

한국어 데이터는 형태소 분석기를 사용하여 토크나이징을 해야 한다고 많은 분이 알고 있습니다. 하지만 여기서는 형태소 분석기가 아닌 위 실습에서 사용했던 내부 단어 토크나이저인 SubwordTextEncoder를 그대로 사용해보세요.

## Step 4. 모델 구성하기

위 실습 내용을 참고하여 트랜스포머 모델을 구현합니다.

## Step 5. 모델 평가하기

Step 1에서 선택한 전처리 방법을 고려하여 입력된 문장에 대해서 대답을 얻는 예측 함수를 만듭니다.